### *Global  Optimization of Portfolios - a rendezvous with   VitaOptimum Plus* !

![](MaxSharpeRatioOptmzn_HeaderImage.png)

![](VOPlus_SharpeRatioOptmzn_HeaderImageCentrCompact.png)

## 1.  Prologue

**Portfolio Optimization** is a discipline that deals with the construction of portfolios from tradable assets such as stocks, bonds and securities, with the dual objectives of ***maximising the expected portfolio return*** and ***minimizing portfolio risk***. The problem models can turn complex when constraints, both linear/non-linear and  representing investor preferences, market norms, investment strategies, why even religious beliefs, are incorporated into them. 

Some of these optimization problem models have been  satisfactorily solved, if not efficiently,  using traditional methods. However, it needs to be realized that traditional methods despite their sophistication,   suffer from pitfalls such as dependence on  a suitable *starting point* or *initial point* in the search space from where the search for solution begins,  or the need to enumerate all sets of candidate solution sets to determine the best or optimal solution, which can be time consuming,  or  the quality or effectiveness of the optimal  solutions to the problems concerned. 

Again, there are problem models which are difficult to be solved by traditional methods. Fortunately,  heuristic methods inspired by nature have helped solve these complex problems to obtain acceptable or near-optimal solutions.  



## 2.  Goal

In this post, we discuss the solution of a fundamental portfolio optimization model, termed **Sharpe Ratio  based portfolio optimization**, which is a non-linear single objective constrained optimization problem.

We solve the model using a traditional method viz., **Sequential Least Squares Quadratic Programming (SLSQP)** using a solver available in **SciPy**, a Python library.

We also solve the same problem using **VitaOptimum Plus**,  a Python based non-traditional solver that employs cutting edge technologies of Artificial Intelligence, Evolutionary Computation, Swarm Intelligence and Advanced Statistics, to arrive at the global optimum. 

The quality of the optimal portfolios obtained by the two solvers are compared and analyzed.   


## 3. Sharpe Ratio

 **Sharpe Ratio**, developed by Nobel Laureate William F Sharpe [SHA 66], is a measure of calculating risk adjusted return. It serves to help investors know about the returns on their investments relative to the risks they hold. The Sharpe Ratio is defined as

![](MaxSharpeRatioOptmznEqn_1.png)
<h5 align="right">..........(1)</h5>


where $r_x$ is the average rate of return on the investment $x$, $R_f$ is  the best available **risk free rate of return** and $\sigma$ is the standard deviation of $r_x$, which denotes the risk on the investment. 

Higher the Sharpe Ratio, more is the excess returns over that of holding a risk free investment, relative to the increased volatility that the investment is exposed to. A Sharpe Ratio of 0, needless to say,  only denotes the investment to be risk-free or one that does not yield any excess return. In practice, while a Sharpe ratio of 1 marks the investment to be acceptable or good for investors, a value less than 1 grades the investment as  sub-optimal,  and values greater than 1 and moving towards 2 or 3, grade the investments as  highly superior. 

## 4.  Maximizing Sharpe Ratio

Having understood the significance of the Sharpe Ratio, let us suppose an investor wishes to make an investment in  assets in such a way that the Sharpe Ratio of the portfolio would be the best possible or the maximum that can be ensured for that investment.  


Let P be a portfolio comprising assets $A_1, A_2, ...A_N$, with $\mu_1, \mu_2, ...\mu_N$ as the asset returns and $W_1, W_2, ...W_N$ as the weights. 

The portfolio return $r$ determined by a weighted summation of its individual asset returns is given by, $\sum\left({W_i.\mu_i}\right)$ and the risk is given by $\sqrt{\sum\sum {W_i.W_j.\sigma_{ij}} } $. (See **Lesson 1 Fundamentals of Risk and Return of a Portfolio** to know about risk and return of a portfolio).  

To keep the discussion simple for now, let us suppose that  the investor decides to enforce only **basic constraints** on the portfolio. (See Sec. 5.2 of **Lesson 5 Mean Variance Optimization Model** to know about basic constraints).  
  
The mathematical model for the Sharpe Ratio based Portfolio Optimization is given by, 

![](MaxSharpeRatioOptmznEqn_2.png)
<h5 align="right">..........(2)</h5>

The numerator of the objective function denotes the excess returns of the investment over that of a risk free asset $R_f$ and the denominator the risk of the investment. The objective is to maximize the Sharpe Ratio. The basic constraints indicate that the investor wishes to have a **fully invested** portfolio, in other words, invest 100% of the capital in the portfolio. 

## 5.  Solving the Sharpe Ratio Optimization Model using *scipy*

To solve the Sharpe Ratio maximization model represented by (2), we make use of the **minimize** library function from **scipy.optimize** package of Python, adopting the *Sequential Least Squares Quadratic Programming* (SLSQP)method.  However, since the original objective function insists on maximization as opposed to minimization demanded by the **minimize** solver, the **principal of  duality** borrowed from Optimization Theory is employed to undertake the transformation. According to the principle,  


![](MaxSharpeRatioOptmznEqn_3.png)
<h5 align="right">..........(3)</h5>

The Python code for the function **MaximizeSharpeRatioOptimization**, which defines the objective function  and the basic constraints represented by equation (2),  is shown here. 

In [21]:
#function to undertake Sharpe Ratio maximization subject to 
#basic constraints of the portfolio

#Solver uses optimize library function from scipy that employs
#SLSQP  (Sequential Leas Squares Quadratic Programming) method to solve the
#non-linear single objective constrained optimization problem

#dependencies
import numpy as np
from scipy import optimize 

def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0.0
    ub = 1.0
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt
    

The Sharpe Ratio optimization requires the computation of risk and return of the portfolio, for which asset returns are required. Function  **StockReturnsComputing** which  computes the asset returns is illustrated below:

In [22]:
# function computes asset returns 
def StockReturnsComputing(StockPrice, Rows, Columns):
    
    import numpy as np
    
    StockReturn = np.zeros([Rows-1, Columns])
    for j in range(Columns):        # j: Assets
        for i in range(Rows-1):     # i: Daily Prices
            StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100

    return StockReturn

## 6.  Case Study

Let us suppose that an investor decides to invest in a portfolio comprising the  following  Dow stocks. While a typical portfolio is a prudent combination of assets comprising equities, bonds, currencies etc., as dictated by the investor's risk appetite, we chose to discuss an equity based portfolio only to keep the narrative simple.  
 
The following are the Dow stocks that has interested the investor:

{Coca-Cola (KO), United Health (UNH), Walt Disney (DIS), IBM (IBM), Cisco (CSCO), JPMorgan Chase (JPM), Goldman Sachs (GS), Walgreens Boots Alliance (WBA), Apple (AAPL), Home Depot (HD), American Express (AXP), McDonald's (MCD), Merck (MRK), Boeing (BA), Caterpillar (CAT)}       

The investor desires to obtain the optimal portfolio set that would yield the maximal Sharpe Ratio.  The objective therefore,  is to find  the optimal weights that will ensure maximal Sharpe Ratio for the portfolio. In other words, the investor wishes to know what proportion of capital must be invested in each of the the stocks so that the portfolio yields maximal Sharpe ratio.

The following Python code reads the dataset DJIA_Apr112014_Apr112019_kpf1.csv (Dow 30: April 11 2014 -April 11 2019) concerned, computes the stock returns using the Python function **StockReturnsComputing** and obtains the mean returns and the variance-covariance matrix of returns. 


In [23]:
#obtain mean and variance-covariance matrix of returns for the portfolio

#Dependencies
import numpy as np
import pandas as pd



#input portfolio dataset comprising 15 stocks
StockFileName = 'DJIA_Apr112014_Apr112019_kpf1.csv'
Rows = 1259  #excluding header
Columns = 15  #excluding date

#read stock prices 
df = pd.read_csv(StockFileName,  nrows= Rows)

#extract asset labels
assetLabels = df.columns[1:Columns+1].tolist()
print('Asset labels of portfolio: \n', assetLabels)

#read asset prices data
StockData = df.iloc[0:, 1:]

#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
print('\nMean Returns:\n', meanReturns)
print('\nVariance-Covariance Matrix of Returns:\n', covReturns)


Asset labels of portfolio: 
 ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'DIS', 'GS', 'HD', 'IBM', 'JPM', 'KO', 'MCD', 'MRK', 'UNH', 'WBA']

Mean Returns:
 [ 0.09   0.029  0.1    0.039  0.081  0.04   0.033  0.085 -0.016  0.06
  0.019  0.057  0.036  0.095 -0.002]

Variance-Covariance Matrix of Returns:
 [[2.375 0.672 0.962 1.042 0.999 0.68  0.954 0.726 0.709 0.825 0.306 0.458
  0.534 0.774 0.697]
 [0.672 1.648 0.8   0.95  0.7   0.569 1.065 0.658 0.663 1.001 0.307 0.35
  0.556 0.718 0.667]
 [0.962 0.8   2.288 1.31  0.89  0.716 1.066 0.747 0.777 0.977 0.381 0.472
  0.578 0.745 0.679]
 [1.042 0.95  1.31  2.733 1.041 0.688 1.321 0.796 0.885 1.169 0.358 0.455
  0.616 0.72  0.681]
 [0.999 0.7   0.89  1.041 1.789 0.713 0.927 0.724 0.817 0.909 0.362 0.477
  0.647 0.656 0.707]
 [0.68  0.569 0.716 0.688 0.713 1.35  0.773 0.586 0.574 0.717 0.302 0.368
  0.466 0.557 0.631]
 [0.954 1.065 1.066 1.321 0.927 0.773 2.114 0.795 0.803 1.554 0.303 0.467
  0.705 0.82  0.819]
 [0.726 0.658 0.747 0.796 0.724 0.586 0

The annual average risk free rate of return in USA during April 2019 was 3%. The daily risk free rate is computed  as   


![](MaxSharpeRatioOptmznEqn_4.png)
<h5 align="right">..........(4)</h5>


The following Python code computes the maximal Sharpe Ratio for  the portfolio. 

In [24]:
#obtain maximal Sharpe Ratio for  the portfolio  of Dow stocks

#set portfolio size
portfolioSize = Columns

#set risk free asset rate of return
Rf=3  # April 2019 average risk  free rate of return in USA approx 3%
annRiskFreeRate = Rf/100

#compute daily risk free rate in percentage
r0 = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
print('\nRisk free rate (daily %): ', end="")
print ("{0:.3f}".format(r0)) 

#initialization
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]
maxSharpeRatio = 0

#compute maximal Sharpe Ratio and optimal weights
result = MaximizeSharpeRatioOptmzn(meanReturns, covReturns, r0, portfolioSize)
print(result)
xOptimal.append(result.x)

    
#compute risk returns and max Sharpe Ratio of the optimal portfolio   
xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 

#set precision for printing results
np.set_printoptions(precision=3, suppress = True)


#display results
print('Maximal Sharpe Ratio: ', maxSharpeRatio, '\nAnnualized Risk (%):  ', \
      annRisk, '\nAnnualized Expected Portfolio Return(%):  ', annRet)
print('\nOptimal weights (%):\n',  xOptimalArray.T*100 )


Risk free rate (daily %): 0.008
     fun: -0.08355172232582905
     jac: array([-0.   ,  0.033, -0.008,  0.044, -0.003,  0.015,  0.046, -0.013,
        0.079,  0.012,  0.012, -0.008,  0.014, -0.016,  0.067])
 message: 'Optimization terminated successfully.'
    nfev: 85
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.137, 0.   , 0.177, 0.   , 0.122, 0.   , 0.   , 0.191, 0.   ,
       0.012, 0.   , 0.137, 0.   , 0.225, 0.   ])
Maximal Sharpe Ratio:  [[1.26]] 
Annualized Risk (%):   [[14.749]] 
Annualized Expected Portfolio Return(%):   [21.584]

Optimal weights (%):
 [[13.694]
 [ 0.   ]
 [17.744]
 [ 0.   ]
 [12.151]
 [ 0.   ]
 [ 0.   ]
 [19.058]
 [ 0.   ]
 [ 1.151]
 [ 0.   ]
 [13.654]
 [ 0.   ]
 [22.547]
 [ 0.   ]]


The output shows that the maximal Sharpe Ratio attainable for the portfolio is 1.26 which is good,  going by practical standards. The annual expected portfolio return is 21.584% against an annualized risk of 14.749%. To achieve this, the optimal capital allocations on the assets of the portfolio are as follows:  

['AAPL': 13. 694%] , ['BA': 17.744%], ['CSCO': 12.151%],  ['HD': 19.058%], ['JPM': 1.151%],  ['MCD': 13.654%], ['UNH': 22.547%].  
  
No investments need  be made in the rest of the assets of the portfolio since the optimal weights arrived at for these assets are 0.  
However, if the investor desires to invest in all the assets in the portfolio, with the weights distributed across all the assets in the portfolio, all that the investor needs to do is to  redefine the bounds constraint of (2) as, $0\lt W_i\lt 1$ and run the **scipy**  solver over the optimization model.

##  7. Solving the Sharpe Ratio Optimization Model using                     *VitaOptimum Plus*

VitaOptimum Plus [VIT, 2019]  is a state-of-the-art global optimization solver encompassing the cutting edge technologies of Artificial Intelligence, Evolutionary Computation, Swarm Intelligence and Advanced Statistics,  to efficiently tackle constrained linear and non-linear models. 
The Sharpe ratio based  portfolio optimization model described in equation (2)and employing the **Ccs** (**C**ontinuous **c**onstrained **s**olver)  Python class from **vitaoptimum.voplus.ccs** package is described below.  

In [25]:
import ctypes
import numpy as np
import pandas as pd

from vitaoptimum.voplus.ccs import Ccs

#global maximum Sharpe Ratio optimization solver wrapper function
def GlobalMaxSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    
    
    # define maximization of Sharpe Ratio using principle of duality
   
    def fobj(x, g, h):
    
        h[0]=sum(x)-1.0 
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func
    
    
    # dimension of the problem and constraints
    dim = PortfolioSize  # problem dimension
    qmeasures = np.zeros (4, dtype = ctypes.c_double)
    nh = 1  # equality constraint dimension [ h(x) = 0 ]
    ng = 0  # non equality constraint dimension [ g(x) <= 0 ]


    # set boundary constraints to [0, 1]
    low = np.zeros(dim, dtype=ctypes.c_double) + [0.001]
    high = np.zeros(dim, dtype=ctypes.c_double) + [1.0]

    solver = Ccs(fobj=fobj,  # objective function with constraints
                 dim=dim, ng=ng, nh=nh,  # dimensions
                 low=low, high=high, qmeasures = qmeasures) # boundary constraints

    results = solver.run()
    
    
    return results


The  optimal weights corresponding to the portfolio of Dow's stocks discussed in Sec. 6 Case Study, is shown below. 

In [26]:
#compute optimal weights and the globally maximal Sharpe Ratio
globlOptimumresult = GlobalMaxSharpeRatioOptmzn(meanReturns, covReturns, r0, portfolioSize)
globlOptimumresult.print()
xOptimal=globlOptimumresult.solution
fitness = globlOptimumresult.best_fobj
print ("optimum: ", xOptimal)



Continuous Constrained Global Optimization Method Result:
Is converged: True

Best function: -0.07615443410286507
Best solution: [0.1161548108071617    0.01038284125499241   0.08436882954464242
 0.003444279712321353  0.00627269742082026   0.0012255520606934869
 0.001870205139497853  0.19377141033854597   0.001058356094975316
 0.008633988856575289  0.17893332696578718   0.06645092573900446
 0.0047187719654376995 0.311317664974979     0.012273123861193546 ]

Quality Measures:
       Delta: 1.0588555827514834e-05
    Diameter: 0.0018545737798590046
   Deviation: 9.111718135881217e-06
  Evaluation: 9286
Best constraint values: [-0.00012321526337199986]
optimum:  [0.1161548108071617    0.01038284125499241   0.08436882954464242
 0.003444279712321353  0.00627269742082026   0.0012255520606934869
 0.001870205139497853  0.19377141033854597   0.001058356094975316
 0.008633988856575289  0.17893332696578718   0.06645092573900446
 0.0047187719654376995 0.311317664974979     0.012273123861193546 ]


The daily and annualized expected portfolio return and the daily and annualized portfolio risk, have been shown below. **maxSharpeRatio** which represents the globally maximum Sharpe ratio has been computed using the formula shown in equation (1).  

In [27]:
#compute risk returns and max Sharpe Ratio of the VitaOptimum Plus optimal portfolio

xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 


#set precision for printing results
np.set_printoptions(precision=2, suppress = True)

#display results
print('\n Maximal Sharpe Ratio: %4.2f' % maxSharpeRatio)
print('\n Annualized Risk %5.2f' % (annRisk), '%')
print('\n Annualized Expected Portfolio Return %5.2f' % (annRet), '%')
print('\n Optimal weights (%):\n',  xOptimalArray.T*100 )




 Maximal Sharpe Ratio: 1.14

 Annualized Risk 13.67 %

 Annualized Expected Portfolio Return 18.56 %

 Optimal weights (%):
 [11.62  1.04  8.44  0.34  0.63  0.12  0.19 19.38  0.11  0.86 17.89  6.65
  0.47 31.13  1.23]


Unlike traditional methods, which yield  a unique optimal solution to the problem model concerned  as decided by the initial point, VitaOptimum Plus, following heuristics exhibits the potential to obtain multiple near-optimal solutions,  during  different runs.   This characteristic of heuristic algorithms  is a boon that can be taken advantage of, while deciding on an effective or practical solution to the problem concerned, from the choices available. 
 

## 8. SciPy solver  Vs VitaOptimum Plus solver

The quality of the optimal portfolios obtained by the **minimize** solver of SciPy  and **Ccs** solver of VitaOptimum Plus,  were studied over the following experiments / observations. 

###  8.1     Experiment 1:  Diversification Ratios of the optimal portfolios

Diversification involves investments in different assets or asset classes or markets. A portfolio that comprises such a diversified set of securities can go a long way in mitigating risk since the securities would react differently to market events. 

A **Diversification Index** quantifies diversification. There are several diversification indices discussed in the literature. **Diversification Ratio** proposed and patented by Yves Choueifaty in 2008 [CHO 08, CHO 13], is a diversification index of recent origin,  built on the inter-dependence between assets of a portfolio. Diversification Ratio is the *ratio of the weighted sum of individual asset volatilities to the portfolio's volatility*. 


Let N be the number of assets in the  portfolio spanning different  asset classes or belonging to a specific class. Let $(\bar{w}=(w_1,w_2,...w_N) )$   be the weights or the proportion of capital to be invested in individual assets in the portfolio and $\bar{w}'$ its transpose. Let $(\bar{\sigma}=(\sigma_1,\sigma_2,...\sigma_N))$ be the standard deviations of returns on the assets and  _V_,  the variance-covariance matrix of returns on the assets. The Diversification Ratio of a portfolio is given as follows:

![](MaxSharpeRatioOptmznEqn_5.png)
<h5 align="right">..........(5)</h5>

**A portfolio that is *most diversified* would yield the *maximal Diversification Ratio*.** 

The following Python code computes the diversification ratios of the optimal portfolios whose weights, variance-covariance matrix and standard deviations of individual asset returns are known.

(Executing the code as it appears in the following cell,  obtains the Diversification Ratio of the portfolio optimized by the VitaOptimum Plus Ccs solver. The code can be used to obtain the Diversification Ratio of the scipy solver obtained portfolio too.) 

In [28]:
# compute diversification ratio of the optimal portfolio
AssetWeights = xOptimalArray
PortfolioRisk = np.sqrt(np.matmul((np.matmul(AssetWeights,\
                             covReturns)), np.transpose(AssetWeights)))

AssetRisk = np.sqrt(np.diagonal(covReturns))
PortfolioDivRatio = sum(np.multiply(AssetRisk, AssetWeights))/PortfolioRisk
print("\n Diversification Ratio %4.2f" % PortfolioDivRatio)


 Diversification Ratio 1.44


Table 1 illustrates the maximal Sharpe Ratio, Annualized Risk(%), Expected Portfolio Return (%) and Diversification Ratio of the optimal portfolio obtained by SciPy and VitaOptimum Plus solvers. 

While SciPy solver delivers a unique optimal solution, VitaOptimum Plus delivers multiple solutions during different runs.

A choice of optimal portfolios delivered by VitaOptimum Plus solver during multiple runs and selected by different investors based on their risk appetites,   in the background of maximal Sharpe Ratios of the portfolios,  have been shown in the table. It can be seen that some investors have opted for minimal risk or maximal diversification ratio portfolios,  while others have opted for maximal return portfolios. The fact that VitaOptimum Plus accommodates such choices due to its potential to deliver   multiple optimal solutions, contributes to its broad based practical appeal and successful application. 

![](MaxSharpeRatioOptmzn_Table1.png)

![](MaxSharpeRatioOptmznTable1.png)

### 8.2     Experiment 2:   Percentage Contributions to Portfolio Risk 

In this experiment,  the percentage contributions to portfolio risk by individual assets in the optimal portfolios,  obtained by SciPy's *minimize* solver and VitaOptimum Plus's Ccs solver were observed. 

The  **Marginal Contribution to Risk**  $\bar{m}$ is given by,

\begin{equation}
\bar{m}=(m_1,m_2,...m_N)'=-\frac{ V.\bar{w'}} { \sqrt{ \bar{w'}. V.\bar{w}}} 
\end{equation}
<h5 align="right">..........(6)</h5>

and $\sigma_P$, the  **portfolio risk** is given by,

\begin{equation}
\sigma_P= \sqrt{ \bar{w'}. V.\bar{w}} 
\end{equation}
<h5 align="right">..........(7)</h5>


The **Absolute Contributions to Total Risk**  is given by
\begin{equation}
w_i.m_i
\end{equation}
<h5 align="right">..........(8)</h5>


and 
 **Percentage contributions to Total Risk**  is  given by

\begin{equation}
\frac{ w_i.m_i} { \sum{w_i.m_i }}
\end{equation}
<h5 align="right">..........(9)</h5>

The python function to compute the percentage contributions to total risk is given below. Here variable mcr represents the marginal contribution to risk computed for all the assets in the portfolio. 
The function needs to be executed over the respective inputs viz., variance-covariance matrix of asset returns, optimal weights and portfolio risk, of the respective portfolios obtained by the two different solvers. 

In [29]:
def totalcontribrisk(covReturns, AssetWeights, PortfolioRisk):
    mcr = np.matmul(covReturns, np.transpose(AssetWeights)) / PortfolioRisk
    tcr = np.around(((np.multiply(mcr, AssetWeights)/PortfolioRisk)),decimals =3) 
    percenttcr = np.asfarray(tcr, float)*100
    return (percenttcr)

Fig. 1 and Fig. 2 illustrate the respective Percentage contributions to total risk computed for the the respective optimal portfolios obtained by the two solvers.  

![](MaxSharpeRatioOptmzn_Fig1.png)
<h4 align="center">Fig.1  SciPy minimize solver portfolio : Percentage Contributions to portfolio  risk <h4/> 

![](MaxSharpeRatioOptmzn_Fig2.png)
<h4 align="center">Fig. 2  VitaOptimum Plus Ccs  solver portfolio : Percentage Contributions to portfolio  risk  <h4/> 

### 8.3     Observation:   Optimal Asset Allocation

The optimal asset allocation represented by the optimal weights and arrived at by SciPy's minimize solver and VitaOptimum Plus's Ccs solver have been shown in Fig (3) and Fig. (4). 

It can be seen that for a Maximal Sharpe Ratio of 1.26 and Diversification Ratio of 1.42, the SciPy solver yields an optimal portfolio where eight of the 15 stocks selected by the investor have been excluded from investing (zero weights), which may not be acceptable to the investor. The practicality of the optimal solution given the investor's preference,  leaves much to be desired. 

On the other hand, VitaOptimum Plus's Ccs solver is able to obtain multifarious optimal asset allocations to appeal to the investors' preferences. Fig. (4) illustrates one such  optimal asset allocaton for a portfolio choice made by an investor opting for minimal risk. It can be seen that the allocation of weights are distributed across assets lending to the practicality of the investment. 

![](MaxSharpeRatioOptmzn_Fig3.png)
<h4 align="center">Fig.3  Optimal Asset Allocation by SciPy's  minimize solver</h4>   

![](MaxSharpeRatioOptmzn_Fig4.png)
<h4 align ="center">Fig.4  Optimal Asset Allocation by VitaOptimum Plus's  Ccs solver </h4>

### 9. Conclusion

SciPy solver indeed yields a unique and distinct optimal portfolio that reports maximal Sharpe Ratio. However, VitaOptimum Plus driven by heuristics, not just delivers multiple optimal solutions that can cater to the risk appetites or preferences of the investors, but also quality portfolios that are less risky, better diversified and well distributed. 

### Companion Reading  

#### Lesson 1 Fundamentals of Risk and Return of a portfolio
https://github.com/PaiViji/PythonFinance-PortfolioOptimization/blob/master/Lesson1_FundaRiskReturnPortfolio/Lesson1_MainContent.ipynb
#### Lesson 3 Heuristic Portfolio Selection
https://github.com/PaiViji/PythonFinance-PortfolioOptimization/blob/master/Lesson3_HeuristicPortfolioSelection/Lesson3_MainContent.ipynb
#### Lesson 5 Mean Variance Optimization Model
https://github.com/PaiViji/PythonFinance-PortfolioOptimization/blob/master/Lesson5_MeanVarianceOptimization/Lesson5_MainContent.ipynb

<h3 align="left">References</h3> 
[CHO 08]   Choueifaty Yves and  Y Coignard, Toward Maximum Diversification, *The Journal of  Portfolio Management*, pp. 40-51, 2008.

[CHO 13]   Choueifaty  Yves, T Froidure and J Reynier, Properties of the Most Diversified Portfolio, *Journal of Investment Strategies*, 2(2), pp. 49-70, 2013.  
 
[SHA 66]  Sharpe, William F. Mutual Fund Performance,  *Journal of Business*, January 1966, pp. 119-138.

[PAI 18]   Vijayalakshmi Pai G. A., Metaheuristics for Portfolio Optimization- An Introduction using MATLAB, Wiley-ISTE, 2018. https://www.mathworks.com/academia/books/metaheuristics-for-portfolio-optimization-pai.html

[VIT, 2019]  http://skyworkflows.com/wp-content/uploads/2019/07/VitaOptimum-by-SkyWorkflows-small.pdf.


